you are given a string
"1:max:4, 2:ann:0, 3:alb:2, 4:edmond:2"

the format of the string is "id:name:manager_id"

Print this: 

Ann
- Alb
- edmond
-- max

In [ ]:
# My Solution using Dicts and DFS

from collections import defaultdict


def printHierarchy(data):
    managerIdToEmployeeId = defaultdict(list)
    idToEmployeeName = {}
    result = []
    
    for data_str in data.split(','):
        data_str = data_str.strip()
        id, name, manager_id = [item.strip() for item in data_str.split(':')]
        managerIdToEmployeeId[manager_id].append(id)
        idToEmployeeName[id] = name
    
    # print(f"managerIdToEmployeeId = {managerIdToEmployeeId}") # {'4': ['1'], '0': ['2'], '2': ['3', '4']})
    # print(f"idToEmployeeName = {idToEmployeeName}") # {'1': 'max', '2': 'ann', '3': 'alb', '4': 'edmond'}
    
    def dfs(idx, prefix):
        if idx not in managerIdToEmployeeId:
            return
        
        subs = managerIdToEmployeeId[idx]
        
        for sub in subs:
            result.append(prefix+idToEmployeeName[sub])
            dfs(sub, prefix+"-")
    
    dfs('0', "")
    
    for res in result:
        print(res)
    
data = "1:max:4, 2:ann:0, 3:alb:2, 4:edmond:2"
printHierarchy(data)

In [ ]:
# Cleaner version of Dicts + DFS approach

from collections import defaultdict


def parse(data):
    managerToEmployees = defaultdict(list)  # manager_id -> [emp_id]
    empIdToName = {}                    # emp_id -> name

    for entry in data.split(','):
        emp_id, name, manager_id = [s.strip() for s in entry.split(':')]
        empIdToName[emp_id] = name
        managerToEmployees[manager_id].append(emp_id)

    return managerToEmployees, empIdToName

def print_hierarchy(managerToEmployees, empIdToName):
    def dfs(emp_id, depth = 0):
        prefix = "-" * depth + (" " if depth > 0 else "")
        print(prefix + empIdToName[emp_id])
        
        for child_id in managerToEmployees.get(emp_id, []):
            dfs(child_id, depth + 1)

    for root_id in managerToEmployees['0']:
        dfs(root_id)

data = "1:max:4, 2:ann:0, 3:alb:2, 4:edmond:2"
# data = "1:max:4, 2:ann:0, 3:alb:2, 4:edmond:2, 5:prem:0"
# data = "5:prem:0, 1:max:4, 2:ann:0, 3:alb:2, 4:edmond:2"
# data = "1:max:4, 2:ann:0, 3:alb:2, 5:prem:0, 4:edmond:2"
managerToEmployees, empIdToName = parse(data)
print_hierarchy(managerToEmployees, empIdToName)

In [ ]:
# Using dataclass for Employee info

from collections import defaultdict
from dataclasses import dataclass, field

@dataclass
class Employee:
    id: str
    name: str
    reports: list = field(default_factory=list)  # direct reports

def parse(data: str) -> dict[str, Employee]:
    employees = {}

    for entry in data.split(','):
        emp_id, name, _ = [s.strip() for s in entry.split(':')]
        employees[emp_id] = Employee(id=emp_id, name=name)

    # second pass to wire up the reporting relationships
    for entry in data.split(','):
        emp_id, _, manager_id = [s.strip() for s in entry.split(':')]
        manager_id = manager_id.strip()
        if manager_id != '0' and manager_id in employees:
            employees[manager_id].reports.append(employees[emp_id])

    return employees

def print_hierarchy(employees: dict[str, Employee]):
    # find roots (employees whose manager_id is '0')
    managed_ids = {report.id for emp in employees.values() for report in emp.reports}
    roots = [emp for emp in employees.values() if emp.id not in managed_ids]

    def dfs(employee: Employee, depth: int = 0):
        prefix = "-" * depth + (" " if depth > 0 else "")
        print(f"{prefix}{employee.name}")
        for report in employee.reports:
            dfs(report, depth + 1)

    for root in roots:
        dfs(root)

data = "1:max:4, 2:ann:0, 3:alb:2, 4:edmond:2"
employees = parse(data)
print_hierarchy(employees)